# Create Heat Map for significant Mismatch Repair genes

This notebook looks at the significant genes in at least one cancer in the DNA Replication pathway. Pancancer heat maps are created with circle size showing significance and color showing differences in median.

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import scipy.stats
import gseapy as gp
import re
import sys 

import cptac
import cptac.utils as u

import plot_utils as p

# Step 1: Run GSEA for significant genes in at least 1 cancer

First read in sig_pval_all_proteins.csv into a df. This csv file contains only genes with a significant p-value in at least one cancer. Then run GSEA using a list of genes from the df.

In [2]:
root = R'~\Github\WhenMutationsDontMatter\PTEN\Step_3_trans_effect\csv'
sig_df = pd.read_csv(root+R'\sig_pval_heatmap.csv')

prot_list = list(sig_df.Proteomics) # list of genes with a sig pval in >= 1 cancer
prot_enr = gp.enrichr(gene_list = prot_list, description='Tumor_partition', gene_sets='KEGG_2016', 
                       outdir='/Enrichr')

In [3]:
prot_enr.res2d.head()

,Term,Overlap,P-value,Adjusted P-value,Old P-value,Old Adjusted P-value,Odds Ratio,Combined Score,Genes,Gene_set
0,Spliceosome Homo sapiens hsa03040,84/134,2.764406e-40,8.099711e-38,0,0,4.683345,426.589830,RBM25;EIF4A3;DDX42;HNRNPU;PRPF19;USP39;EFTUD2;...,KEGG_2016
1,RNA transport Homo sapiens hsa03013,76/172,3.295014e-23,4.827196e-21,0,0,3.301161,170.891381,CYFIP2;NUP107;NUP188;RPP30;EIF4A3;PHAX;PNN;XPO...,KEGG_2016
2,DNA replication Homo sapiens hsa03030,22/36,3.214005e-11,3.139012e-09,0,0,4.565641,110.310090,RFC5;FEN1;RFC3;PCNA;RFC4;MCM7;RFC1;RFC2;PRIM1;...,KEGG_2016
3,Mismatch repair Homo sapiens hsa03430,16/23,9.814851e-10,7.189378e-08,0,0,5.197252,107.801163,RFC5;RFC3;PCNA;RFC4;RFC1;RFC2;RPA1;MLH1;POLD3;...,KEGG_2016
4,Ribosome biogenesis in eukaryotes Homo sapiens...,34/89,3.894836e-09,2.282374e-07,0,0,2.854109,55.265863,NVL;WDR3;RPP30;HEATR1;NAT10;NMD3;PWP2;WDR43;NO...,KEGG_2016


# Step 2: Get the list of significant genes 

In [4]:
index = 4
trans = prot_enr.res2d.Genes[index]
genes = trans.split(';')
print(prot_enr.res2d.Term[index])
print('total genes:',len(genes))

Ribosome biogenesis in eukaryotes Homo sapiens hsa03008
total genes: 34


# Step 3: Create HeatMap

Slice out genes from the DNA Replication pathway from the df with genes sig in > 1 cancer.

In [5]:
# sig > 1 cancer
bool_df = sig_df.Proteomics.isin(genes)
plot_df = sig_df[bool_df]
len(plot_df.Proteomics.unique())

34

In [6]:
p.plotCircleHeatMap(plot_df, circle_var = 'P_Value', color_var='Medians', x_axis= 'Proteomics', y_axis = 'Cancer',
                    plot_height=400, plot_width= 800)

C:\Users\brittany henderson\Anaconda3\envs\Research\lib\site-packages\plot_utils\__init__.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["size2"] = df[circle_var].apply(lambda x: -1*(np.log(x)))
C:\Users\brittany henderson\Anaconda3\envs\Research\lib\site-packages\plot_utils\__init__.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['size'] = (df["size2"])*3


Loading BokehJS ...

# Step 4: Create a HeatMap with both +/- differences in median

 Read in the pos_neg_df.csv to create a df with only genes that have a pos and neg difference in median in different cancers. Slice out the genes that have a pos and neg difference in median in the pathway using the list of genes with a significant p-value in the pathway. 

In [7]:
pos_neg_df = pd.read_csv(root+R'\pos_neg_df.csv')

In [8]:
get = pos_neg_df.Proteomics.isin(genes) # bool df where True has both pos and neg
genes_pn = pos_neg_df[get] # Keep only genes with pos and neg
genes_pn.Proteomics.unique()

array(['WDR75', 'HEATR1', 'NAT10', 'IMP3', 'NOB1', 'DKC1', 'MPHOSPH10',
       'UTP6'], dtype=object)

In [9]:
p.plotCircleHeatMap(genes_pn, circle_var = 'P_Value', color_var='Medians', x_axis= 'Proteomics', y_axis = 'Cancer',
                    plot_height=400, plot_width=600)

Loading BokehJS ...

# Sig in multiple cancers

In [10]:
mult_sig_df = pd.read_csv(root+R'\mult_sig_pval_heatmap.csv')

mult_sig_list = list(mult_sig_df.Proteomics.unique()) # list of genes with a sig pval in > 1 cancer
enr2 = gp.enrichr(gene_list = mult_sig_list, description='Tumor_partition', gene_sets='KEGG_2016', 
                       outdir='/Enrichr')

In [11]:
enr2.res2d.head()

,Term,Overlap,P-value,Adjusted P-value,Old P-value,Old Adjusted P-value,Odds Ratio,Combined Score,Genes,Gene_set
0,Ribosome biogenesis in eukaryotes Homo sapiens...,13/89,4.277725e-09,1.253373e-06,0,0,8.443203,162.699213,UTP6;IMP3;WDR3;HEATR1;NAT10;WDR75;IMP4;PWP2;UT...,KEGG_2016
1,DNA replication Homo sapiens hsa03030,9/36,7.804590e-09,1.143372e-06,0,0,14.450867,269.776790,RFC3;PCNA;RFC4;MCM7;RFC2;MCM3;MCM4;MCM5;MCM2,KEGG_2016
2,RNA transport Homo sapiens hsa03013,17/172,8.312239e-09,8.118287e-07,0,0,5.713133,106.295915,RANBP2;NUP210;NUP155;NCBP1;NUP133;THOC3;NUP153...,KEGG_2016
3,B cell receptor signaling pathway Homo sapiens...,11/73,4.843587e-08,3.547928e-06,0,0,8.710112,146.704630,PPP3CB;SYK;CHUK;PIK3CA;PRKCB;INPP5D;BLNK;RAC2;...,KEGG_2016
4,Fc gamma R-mediated phagocytosis Homo sapiens ...,12/93,7.049700e-08,4.131124e-06,0,0,7.458512,122.824507,PAK1;PTPRC;SYK;PIK3CA;MYO10;PRKCB;INPP5D;RAC2;...,KEGG_2016


In [12]:
index2 = 4
trans2 = enr2.res2d.Genes[index2]
genes_mult = trans2.split(';')
print(enr2.res2d.Term[index2])
print('total genes:',len(genes_mult))

Fc gamma R-mediated phagocytosis Homo sapiens hsa04666
total genes: 12


In [13]:
bool_df = mult_sig_df.Proteomics.isin(genes_mult)
plot_df2 = mult_sig_df[bool_df]
plot_df2.Proteomics.unique()

array(['DOCK2', 'SYK', 'PRKCB', 'PLCG2', 'PIK3CD', 'INPP5D', 'MYO10',
       'PAK1', 'WAS', 'PTPRC', 'RAC2', 'PIK3CA'], dtype=object)

In [14]:
p.plotCircleHeatMap(plot_df2, circle_var = 'P_Value', color_var='Medians', x_axis= 'Proteomics', y_axis = 'Cancer',
                    plot_height=400, plot_width=800)

C:\Users\brittany henderson\Anaconda3\envs\Research\lib\site-packages\plot_utils\__init__.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["size2"] = df[circle_var].apply(lambda x: -1*(np.log(x)))
C:\Users\brittany henderson\Anaconda3\envs\Research\lib\site-packages\plot_utils\__init__.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['size'] = (df["size2"])*3


Loading BokehJS ...

In [15]:
get = pos_neg_df.Proteomics.isin(genes_mult) # bool df where True has both pos and neg
genes_pn_mult = pos_neg_df[get] # Keep only genes with pos and neg
genes_pn_mult.Proteomics.unique()

array(['SYK', 'PLCG2', 'MYO10', 'PAK1'], dtype=object)

In [16]:
p.plotCircleHeatMap(genes_pn_mult, circle_var = 'P_Value', color_var='Medians', x_axis= 'Proteomics', y_axis = 'Cancer',
                    plot_height=400, plot_width=600)

Loading BokehJS ...

# Check other pathways the genes (+/- and sig in mult. cancers) are part of

In [17]:
gnm = list(genes_pn_mult.Proteomics.unique())
print(gnm)
enr3 = gp.enrichr(gene_list = gnm, description='Tumor_partition', gene_sets='KEGG_2016', 
                       outdir='/Enrichr')

['SYK', 'PLCG2', 'MYO10', 'PAK1']


In [18]:
enr3.res2d.head()

,Term,Overlap,P-value,Adjusted P-value,Old P-value,Old Adjusted P-value,Odds Ratio,Combined Score,Genes,Gene_set
0,Fc gamma R-mediated phagocytosis Homo sapiens ...,4/93,4.379947e-10,1.283324e-07,0,0,215.053763,4634.153627,PAK1;SYK;MYO10;PLCG2,KEGG_2016
1,Natural killer cell mediated cytotoxicity Homo...,3/135,1.197103e-06,1.753755e-04,0,0,111.111111,1515.067386,PAK1;SYK;PLCG2,KEGG_2016
2,Epithelial cell signaling in Helicobacter pylo...,2/68,6.804004e-05,6.645244e-03,0,0,147.058824,1411.090318,PAK1;PLCG2,KEGG_2016
3,Fc epsilon RI signaling pathway Homo sapiens h...,2/68,6.804004e-05,4.983933e-03,0,0,147.058824,1411.090318,SYK;PLCG2,KEGG_2016
4,B cell receptor signaling pathway Homo sapiens...,2/73,7.846784e-05,4.598216e-03,0,0,136.986301,1294.907079,SYK;PLCG2,KEGG_2016


In [19]:
# Look at p-values 
certain_gene = 'PCNA'
all_df = pd.read_csv(root+R'\all_heatmap.csv')
gene_df = all_df.loc[all_df['Proteomics'] == certain_gene]
gene_df

,Proteomics,P_Value,Medians,Cancer
131,PCNA,0.001534,0.631486,Gbm
11320,PCNA,0.026069,0.420740,Hnscc
21556,PCNA,0.157360,0.501850,Luad
32644,PCNA,0.999914,-0.070550,Lscc
41897,PCNA,0.598944,0.361150,Brca
49311,PCNA,0.985666,0.028375,Ov
60424,PCNA,0.118831,-0.235000,En
70335,PCNA,0.781963,-0.142000,Colon
